<a href="https://colab.research.google.com/github/nelmsal/MUSA650_FinalProject_RightOfWayClassification/blob/main/FinalProject_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling

## Import Packages & Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import pandas as pd
import json
import numpy as np

PROJ_DIR = 'drive/MyDrive/Penn/MUSA-650/FinalProject'

%cd {PROJ_DIR}

/content/drive/MyDrive/Penn/MUSA-650/FinalProject


### Image, Mask, & Information Dataframes

In [4]:
#DATA_DIR = PROJ_DIR + '/' + 'data'
DATA_DIR = 'data'

IMAGE_PATH = DATA_DIR + '/clean/' + 'sf_naip_images.parquet'
MASK_PATH = DATA_DIR + '/clean/' + 'sf_naip_mask.parquet'
INFO_PATH = DATA_DIR + '/clean/' + 'sf_naip_info.parquet'

In [5]:
!python -m pip install 'fsspec>=0.3.3'

In [6]:
def df_to_array(arr_df):
  # import csv of image data 
  ## pre-flattened
  cols = [col for col in list(arr_df) if 'Unnamed' not in col]
  arr_df = arr_df[cols]

  # list of shape index colnames
  icols = [col for col in list(arr_df) if 'ass' not in col]
  # make array out of rgb dataframe
  #image_arr = np.stack(arr_df[icols].apply(list, axis=1))
  image_arr = arr_df[icols].to_numpy(dtype='int')
  print('Shape: {}'.format(image_arr.shape))

  return image_arr


In [7]:
import dask.dataframe as dd

img_dd = dd.read_parquet(
             IMAGE_PATH,
             blocksize=1000000,
             sample=655360,
             dtype='int'
             )

In [8]:
img_arr = df_to_array(img_dd.compute(scheduler='processes'))

Shape: (18811, 65536)


In [ ]:
#mask_df = pd.read_csv(MASK_PATH)
mask_arr = df_to_array(pd.read_parquet(MASK_PATH))
info_df = pd.read_parquet(INFO_PATH)

### Prepare Data

In [10]:
img_shape = (128, 128, 4)
mask_shape = (128, 128, 1)
rgb_shape = (128,128,3)

# get shape
def unflatten(array, new_shape):
  # shape initial
  n_samples = array.shape[0]
  flat_shape = array.shape[1]

  new_shape = [n_samples] + list(new_shape)

  return array.reshape(new_shape)

### Train / Test Split

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam, RMSprop
import keras

# load vgg model
from keras.applications.vgg16 import VGG16, preprocess_input

# SPLIT GREY DATAFRAME INTO 50/50 
## WITH STRATIFICATION ON THE CLASSES COL
X_train, X_test, y_train, y_test = train_test_split(
    img_arr, mask_arr, 
    test_size=0.75, random_state=420
    )
## Scale the data
scalar = MinMaxScaler()
scalar.fit(X_train)
#X_train = scalar.transform(X_train)
#X_test = scalar.transform(X_test)
X_train = unflatten(X_train, img_shape)[:,:,:,:3]
#X_train = preprocess_input(X_train)
X_test = unflatten(X_test, img_shape)[:,:,:,:3]
#X_test = preprocess_input(X_test)
y_train = unflatten(y_train, mask_shape)
y_test = unflatten(y_test, mask_shape)

input_shape = X_train.shape[1:]
print('X train: {}'. format(X_train.shape))
print('Y train: {}'. format(y_train.shape))

X train: (4702, 128, 128, 3)
Y train: (4702, 128, 128, 1)


## Model

### Import VGG16

In [ ]:

# load the model
vgg16 = VGG16(
    include_top = False,
    input_shape = rgb_shape,
    weights='imagenet'
)
vgg16.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import \
  Conv2D, Conv2DTranspose, MaxPooling2D, \
  Dense, Activation, Dropout, \
  Flatten, Input, Concatenate, \
  BatchNormalization
from tensorflow.keras.initializers import RandomNormal

def conv_block(inputs,num_filters):
  x = Conv2D(num_filters,3,padding='same')(inputs)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(num_filters,3,padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x) 
  return x

def define_decoder(inputs,skip_layer,num_filters):
  init = RandomNormal(stddev=0.02)
  x = Conv2DTranspose(
        num_filters,(2,2),
        strides=(2,2),
        padding='same',
        kernel_initializer=init
      )(inputs)  
  g = Concatenate()([x,skip_layer])
  g = conv_block(g, num_filters)
  return g

def vgg16_unet(input_shape):
  inputs = Input(shape=input_shape)
  vgg16 = VGG16(include_top=False,weights='imagenet',input_tensor=inputs)  
  # We will extract encoder layers based on their output shape from vgg16 model  
  s1 = vgg16.get_layer('block1_conv2').output  
  s2 = vgg16.get_layer('block2_conv2').output  
  s3 = vgg16.get_layer('block3_conv3').output  
  s4 = vgg16.get_layer('block4_conv3').output    # bottleneck/bridege layer from vgg16
  b1 = vgg16.get_layer('block5_conv3').output #32
  
  # Decoder Block
  d1 = define_decoder(b1,s4,512)
  d2 = define_decoder(d1,s3,256)
  d3 = define_decoder(d2,s2,128)
  d4 = define_decoder(d3,s1,64)  #output layer
  outputs = Conv2D(1,1, padding='same', activation='sigmoid')(d4)
  model = Model(inputs,outputs)
  
  return model

### Import Metrics

In [ ]:
from tensorflow.math import reduce_sum

# Dice Loss 
## a measure of overlap between two samples
## ranges from 0 to 1 
## a Dice coefficient of 1 denotes perfect and complete overlap
smooth = 1e-15
def dice_coef(y_true,y_pred):
  y_true = Flatten()(y_true)
  y_pred = Flatten()(y_pred)
  intersection = reduce_sum(y_true*y_pred)
  return (2. * intersection + smooth) / (reduce_sum(y_true) + reduce_sum(y_pred))
  
def dice_loss(y_true,y_pred):
  return 1.0 - dice_coef(y_true,y_pred)

# Intersection-Over-Union 
## a common evaluation metric for semantic image segmentation
## true_positive / (true_positive + false_positive + false_negative)
def iou(y_true,y_pred):
  def f(y_true,y_pred):
    intersection = (y_true*y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    x = (intersection + 1e-15) / (union + 1e-15)
    x = x.astype(np.float32)
    return x
  return tf.numpy_function(f,[y_true,y_pred],tf.float32)

### Compile Model

In [15]:
from tensorflow.keras.metrics import Precision, Recall

model = vgg16_unet(rgb_shape)
model.compile(
    loss=dice_loss,
    optimizer=Adam(lr = .001),
    metrics=[dice_coef, iou, Recall(), Precision()]
    )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### Fit Model

In [16]:
tf.config.run_functions_eagerly(True)

In [ ]:
EPOCHS = 25
#train_steps = len(X_train)//batch_size
#test_steps = len(X_test)//batch_size

history = model.fit(
  X_train, tf.cast(y_train, tf.float32),
  epochs=EPOCHS,
  validation_data=(X_test, tf.cast(y_test, tf.float32))#,
  #steps_per_epoch=train_steps,
  #validation_steps=test_steps
  )

# convert the history.history dict to a pandas DataFrame:     

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/25
147/147 [==============================] - 105s 652ms/step - loss: 0.4033 - dice_coef: 0.5967 - iou: 0.4375 - recall: 0.7591 - precision: 0.4856 - val_loss: 0.3248 - val_dice_coef: 0.6752 - val_iou: 0.5114 - val_recall: 0.8827 - val_precision: 0.5744
Epoch 2/25
147/147 [==============================] - 85s 582ms/step - loss: 0.2305 - dice_coef: 0.7696 - iou: 0.6271 - recall: 0.7877 - precision: 0.7646 - val_loss: 0.3496 - val_dice_coef: 0.6504 - val_iou: 0.4844 - val_recall: 0.5092 - val_precision: 0.9220
Epoch 3/25
147/147 [==============================] - 85s 580ms/step - loss: 0.1896 - dice_coef: 0.8104 - iou: 0.6828 - recall: 0.8107 - precision: 0.8168 - val_loss: 0.1882 - val_dice_coef: 0.8118 - val_iou: 0.6846 - val_recall: 0.8201 - val_precision: 0.8118
Epoch 4/25
147/147 [==============================] - 85s 579ms/step - loss: 0.1735 - dice_coef: 0.8265 - iou: 0.7059 - recall: 0.8234 - precision: 0.8358 - val_loss: 0.2257 - val_dice_coef: 0.7743 - val_iou: 0.6336 

In [ ]:
import datetime 

now = datetime.datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M")

model_path = os.path.join(DATA_DIR, 'models', 'vgg16_model_{}.keras'.format(dt_string))
model.save(model_path)

hist_df = pd.DataFrame(history.history)
history_path = os.path.join(DATA_DIR, 'models', 'vgg16_history_{}.csv'.format(dt_string))
with open(history_path, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
reconstructed_model.history

## Results

### Accuracy

In [ ]:
print(model.history.history)

In [ ]:
def plot_loss_acc(fitted_model, num_epoch = EPOCHS):
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots(1,2, figsize=[18,6])
  ax[0].plot(range(1, num_epoch+1), fitted_model.history['loss'], c='blue', label='Training loss')
  ax[0].plot(range(1, num_epoch+1), fitted_model.history['val_loss'], c='red', label='Validation loss')
  ax[0].legend()
  ax[0].set_xlabel('epochs')

  ax[1].plot(range(1, num_epoch+1), fitted_model.history['accuracy'], c='blue', label='Training accuracy')
  ax[1].plot(range(1, num_epoch+1), fitted_model.history['val_accuracy'], c='red', label='Validation accuracy')
  ax[1].legend()
  ax[1].set_xlabel('epochs')

plot_loss_acc(model, EPOCHS)
#classification_accuracy = max(model.history['accuracy'])

In [ ]:
import matplotlib.pyplot as plt
import random
from skimage.io import imread
import matplotlib
from matplotlib import colors

# counts array equal to focus value
def count_focus(array, value=1):
    return len(array[array==value])
# get percent that the array is equal to value
## used for filter masks
def get_pct_arr(focus_array, value=1):
    return np.array([count_focus(arr, value=value)/arr.size for arr in focus_array])

smooth = 1e-15
def dice_coef_arr(y_true,y_pred):
  y_true = np.array(y_true).flatten()
  y_pred = np.array(y_pred).flatten()
  intersection = reduce_sum(y_true*y_pred)
  return (2. * intersection + smooth) / (reduce_sum(y_true) + reduce_sum(y_pred))
  
def dice_loss_arr(y_true,y_pred):
  return 1.0 - dice_coef(y_true,y_pred)

def arr_to_binary(arr, cutoff=1.0, buffer=.01, good=1, bad=0):
  arr[arr>=cutoff-buffer] = good
  arr[arr<cutoff-buffer] = bad
  return arr

def get_masked_cmap(arr, thresh = 1.0, cmap='red'):
  masked_array = np.ma.masked_where(arr < thresh, arr)
  cmap = colors.ListedColormap([cmap])
  return masked_array, cmap

cols, rows = 4, 5
axes = []
[[axes.append((c,r)) for r in range(rows)] for c  in range(cols)]
axis_count = len(axes)
fig, ax = plt.subplots(rows, cols, figsize=[12, 16])

for row, idx in enumerate(random.sample(range(len(X_test)), rows)):

    focus_image, observed_mask = X_test[idx:idx+1,:,:,:], y_test[idx:idx+1,:,:,:]
    
    predicted_mask = model.predict(tf.cast(focus_image, tf.float32))

    predicted_mask_thresh = arr_to_binary(predicted_mask.copy(), buffer=0.01)
    dice_loss = dice_loss_arr(tf.cast(observed_mask, tf.float32), predicted_mask_thresh)
    
    pct_observed = get_pct_arr(observed_mask)[0]
    pct_predicted = get_pct_arr(predicted_mask)[0]
    pct_predicted_thresh = get_pct_arr(predicted_mask_thresh)[0]

    ax[row, 0].imshow(focus_image[0])
    ax[row, 0].set_title('RGB Image' + '\n' + 'Dice Loss: {:.2f}'.format(dice_loss))

    ax[row, 1].imshow(focus_image[0])
    ax[row, 1].imshow(observed_mask.reshape(128,128), alpha=.5)
    ax[row, 1].set_title('Right-of-Way Mask' + '\n' + 'Observed Pct: {0:.0%}'.format(pct_observed))
    

    
    ax[row, 2].imshow(focus_image[0])
    ax[row, 2].imshow(predicted_mask.reshape(128,128), alpha=.5)

    diff_mask = arr_to_binary(predicted_mask - observed_mask, buffer=0.01, bad=np.nan).reshape(128,128)
    diff_mask, cmap = get_masked_cmap(diff_mask, cmap='green')
    ax[row, 2].imshow(diff_mask, cmap=cmap)
    diff_mask  = arr_to_binary(observed_mask - predicted_mask, buffer=0.01, bad=np.nan).reshape(128,128)
    diff_mask, cmap = get_masked_cmap(diff_mask, cmap='red')
    ax[row, 2].imshow(diff_mask, cmap=cmap)
    #ax[row, 2].set_title('Pct Roads: {0:.1%}'.format(pct_roads) + '\n' + 'Pct Diff: {0:.1%}'.format(pct_diff))
    ax[row, 2].set_title('Predicted Mask' + '\n' + 'Predicted Pct: {0:.0%}'.format(pct_predicted))
    
    

    ax[row, 3].imshow(focus_image[0])
    ax[row, 3].imshow(predicted_mask_thresh.reshape(128,128), alpha=.5)

    diff_mask = arr_to_binary(predicted_mask_thresh - observed_mask, buffer=0.01, bad=np.nan).reshape(128,128)
    diff_mask, cmap = get_masked_cmap(diff_mask, cmap='green')
    ax[row, 3].imshow(diff_mask, cmap=cmap)
    diff_mask  = arr_to_binary(observed_mask - predicted_mask_thresh, buffer=0.01, bad=np.nan).reshape(128,128)
    diff_mask, cmap = get_masked_cmap(diff_mask, cmap='red')
    ax[row, 3].imshow(diff_mask, cmap=cmap)
    #ax[row, 3].imshow(predicted_mask_thresh.reshape(128,128), alpha=.5)
    ax[row, 3].set_title('Predicted Mask (p>1)' + '\n' + 'Predict Thresh Pct: {0:.0%}'.format(pct_predicted_thresh))

    for col in range(cols):
        ax[row, col].set_axis_off()
fig.patch.set_facecolor('#FFFFFF')

In [ ]:
arr = arr_to_binary(observed_mask - predicted_mask_thresh, buffer=0, bad=0).reshape(128,128)
thresh=1
np.ma.masked_where(arr < thresh, arr)

In [ ]:
diff_mask[~np.isnan(diff_mask)]

In [ ]:
smooth = 1e-15
def dice_coef(y_true,y_pred):
  y_true = np.array(y_true).flatten()
  y_pred = np.array(y_pred).flatten()
  intersection = reduce_sum(y_true*y_pred)
  return (2. * intersection + smooth) / (reduce_sum(y_true) + reduce_sum(y_pred))
  
def dice_loss(y_true,y_pred):
  return 1.0 - dice_coef(y_true,y_pred)

dice_loss(tf.cast(focus_image, tf.float32), predicted_image)

In [ ]:
X_test